In [ ]:
from google.colab import files
uploaded = files.upload()  # lets you pick files from your PC

In [ ]:
#import data
import pandas as pd
import numpy as np
import importlib
import glob


#transfer_plate=pd.read_csv("test/transfer_plate.csv",header=0)
arr = pd.read_csv("/content/collab_data/collab_data/test/transfer_plate.csv").iloc[:, 1:-4].to_numpy()
transfer_plate_labels=pd.read_csv("/content/collab_data/collab_data/test/transfer_plate.csv").iloc[:, -4:].to_numpy()
test_data = pd.read_csv("/content/collab_data/collab_data/test/96_samples.csv",header=None).iloc[:, 1:].to_numpy()

csv_files = glob.glob("/content/collab_data/collab_data/train/*.csv")
dataframes = {}
for file in csv_files:

    dataframes[file] = pd.read_csv(file)


In [ ]:
transfer_plate_labels_1=[]
for i in range(len(transfer_plate_labels)-1):

    transfer_plate_labels_1.append(transfer_plate_labels[i])
    transfer_plate_labels_1.append(transfer_plate_labels[i])

transfer_plate_labels_1=np.array(transfer_plate_labels_1)[:192,-3:]
transfer_plate_labels_1=np.array(transfer_plate_labels_1,dtype=np.float32)


In [ ]:
from scipy.interpolate import interp1d

target_wavenumbers = np.linspace(300, 1942, 1340)
interpolated = []
conc=[]

for file in csv_files:
    # Convert column names to float

    wavelengths = dataframes[file].columns[:-5].astype(float)
    # Get only the spectral data
    intensities_y = (dataframes[file].values)

    conc.append(intensities_y[:,-5:-2])
    #normalization steps --1-- scaling

    intensities_y=intensities_y/(np.max(intensities_y))



    # Interpolate each row (spectrum)

    for row in intensities_y:

        f = interp1d(wavelengths, row[:-5], kind='quadratic', fill_value='extrapolate')

        interpolated.append(f(target_wavenumbers))



interpolated=np.array(interpolated)

#normalization step --2--

interpolated=(interpolated-np.mean(interpolated))/np.std(interpolated)

conc=np.vstack(conc)


In [ ]:
arr = np.char.strip(arr.astype(str), "[]").astype(float)
test_data = np.char.strip(test_data.astype(str),"[]").astype(float)

# normalization step --- 1 --- scaling
arr=arr/np.max(arr)

test_data=test_data/np.max(test_data)


In [ ]:
wavelengths=np.linspace(65,3350,2048)#make sure this is the actual interval.

#interpolate test_data and the transfer_plate datasets(same format)
transfer_plate=[]
clean_test=[]
for row in arr:

    f = interp1d(wavelengths, row, kind='quadratic', fill_value='extrapolate')
    transfer_plate.append(f(target_wavenumbers))

for row in test_data:

    g = interp1d(wavelengths, row, kind='quadratic', fill_value='extrapolate')

    clean_test.append(g(target_wavenumbers))


transfer_plate=(transfer_plate-np.mean(transfer_plate))/np.std(transfer_plate)
clean_test=(clean_test-np.mean(clean_test))/np.std(clean_test)


transfer_plate=np.array(transfer_plate,dtype=np.float32)

In [ ]:
import Data_augment as Da
importlib.reload(Da)

train_data=np.concatenate((transfer_plate,interpolated))
labels=np.concatenate((transfer_plate_labels_1,conc))
labels_scaled,labels_max= Da.reversible_scaler(labels,None)

labels_max

0
0


array([16.7344 ,  3.05065,  3.94541])

In [ ]:
from Raman_Dataset import RamanDataset
from torch.utils.data import DataLoader
import Data_augment as Da
importlib.reload(Da)



dataset1 = RamanDataset(train_data, labels_scaled)




loader = DataLoader(dataset1, batch_size=32, shuffle=True)

# Example: iterate
for spectra_batch, targets_batch in loader:
    print(spectra_batch.shape)  # (batch_size, 1, num_points)

    print(targets_batch.shape)  # (batch_size, 3)
    break

0
0
torch.Size([32, 1, 1340])
torch.Size([32, 3])


-------Big Model training ------

In [ ]:
import SpectraCNN as CNN
import torch
import torch.nn as nn
importlib.reload(CNN)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


model = CNN.SpectraCNN()

model = model.to(device) #move to gpu if available

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

epochs=100

for epoch in range(epochs):
    running_loss=0
    print(epoch)
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch}: Loss {running_loss/len(loader)}")

In [ ]:
clean_test=np.array(clean_test,dtype=np.float32)

In [ ]:
test_loader = DataLoader(RamanDataset(clean_test,None, train=False), batch_size=32, shuffle=False)


model.eval()  # switch to evaluation mode
predictions = []

with torch.no_grad():
    for spectra in test_loader:
        outputs = model(spectra)  # shape (batch, 3)
        predictions.append(outputs)

predictions = torch.cat(predictions).numpy()
predictions2=Da.reversible_scaler(predictions,labels_max,reverse=True)

Cross validation for model tuning and ensembling

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import KFold
from SpectraCNN import SpectraCNN


import Cross_validation as cv
importlib.reload(cv)

train_loss, val_loss = cv.run_kfold_cv(train_data, labels_scaled, k=5, batch_size=32, epochs=10, model_class = SpectraCNN)


---- Fold 1/5 ----
Using device: cpu


KeyboardInterrupt: 